<a href="https://colab.research.google.com/github/stephenbeckr/numerical-analysis-class/blob/master/Demos/Ch1_DataTypes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic practice with datatypes and overflow/underflow etc. in Python

Datatypes in Python.
1. Integers
2. Floating point
3. Complex
4. Boolean

See https://www.math.ubc.ca/~pwalls/math-python/python/numbers/

Some oldschool information at [What every computer scientist should know about floating-point arithmetic (1991)](https://docs.oracle.com/cd/E19957-01/806-3568/ncg_goldberg.html)

See this [decimal-to-floating point converter](https://www.exploringbinary.com/floating-point-converter/) which lets you see if a number can be exactly represented by a floating point number (check if the "inexact" flag is checked). For eample, `0.5` can be exactly stored (as can integers), but `0.4` cannot be exactly stored. There are [other ways to check mentioned on the internet](https://stackoverflow.com/a/28648065) and [python packages that do this](https://stackoverflow.com/a/8458835).

In [ ]:
type( 3 )

int

In [ ]:
type( 3.0 )

float

In [ ]:
print( 11 % 4 )  # remainder (modulo)
print( 11 // 4 ) # integer division
print( 11 / 4 )  # in Python3, this is always a float
print( 4*( 11//4 ) + 11%4 )

3
2
2.75
11


Scientific notation. Use `e` or `E`

In [ ]:
1e-5

Complex numbers

In [ ]:
print( complex(1,2) )
type(  complex(1,2) )

type( 4 - 7j )

(1+2j)


complex

Rounding

In [ ]:
import math
print( math.floor( math.pi) )
print( round( math.pi, 6 ) )

3
3.141593


In [ ]:
# We can request 30 digits, but only about 16 are available
print( round( math.pi, 30 ) )

3.141592653589793


Be careful with copies!

In [ ]:
# Be careful! Python (and NumPy) assign by reference. They do not make copies.  
# Matlab does this (lazily), so it's easy for Matlab users to have bugs in Python
x = [3, 4, 5]
y = x
x[0] = 30
y

In [ ]:
x = [3, 4, 5]
y = x.copy()   # we must explicitly ask Python to copy the data
x[0] = 30
y

## NumPy
See [NumPy for Matlab users](https://numpy.org/doc/stable/user/numpy-for-matlab-users.html)

In [ ]:
import numpy as np

myArray = np.arange(10)

In [ ]:
myArray

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
type( myArray )

numpy.ndarray

In [ ]:
myArray.dtype

dtype('int64')

In [ ]:
myArray[2] = -10.5
myArray  # it automatically rounded to an integer

array([  0,   1, -10,   3,   4,   5,   6,   7,   8,   9])

In [ ]:
myNewArray = myArray.astype( np.float64 ) # this makes a new copy
myNewArray.dtype

dtype('float64')

 NumPy has datatypes like:
 - `bool` (Boolean: a single bit, either `True` or `False`)
 - `int8` and `uint8`
 - ...
 - `int64` and `uint64`
 - `float16` (half precision)
 - `float32` (single precision, the standard a long time ago)
 - `float64` (double precision; the standard now)
 - `float128` (extended-precision)
 - `complex64`, `complex128`, `complex256`
 - `string_` or `unicode_`


In [ ]:
myNewArray

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [ ]:
myArray

array([ 0,  1, 20,  3,  4,  5,  6,  7,  8,  9])

In [ ]:
np.finfo( float )  # `float` is shorthand for `float64`

finfo(resolution=1e-15, min=-1.7976931348623157e+308, max=1.7976931348623157e+308, dtype=float64)

In [ ]:
machineEpsilon = np.finfo( float ).eps  # see https://www.kite.com/python/docs/numpy.finfo
# Same as np.spacing(np.float64(1))
machineEpsilon

In [ ]:
2**(-52)

In [ ]:
tiny = np.finfo( float ).tiny
# smallest "normal" number, e.g., mantissa isn't too small, but exponent is maxed out
tiny

In [ ]:
mn, mx = np.finfo( float ).min, np.finfo( float ).max
mn, mx

The mn/max look very similar to the `tiny` variable, but that's misleading, because they are different kinds of quantities.

In [ ]:
1 + machineEpsilon/2 == 1

True

In [ ]:
# Overflow
print( (10 + mx) - mx )

# Overflow can have negative values
print( (10 - mn) + mn )

0.0
0.0


In [ ]:
# Underflow is for very small numbers **in absolute value**
# Smallest "subnormal" number, changing mantissa from 1 + ... to just ...
veryTiny = np.spacing(np.float64(0)) # not the same as "tiny"; see https://stackoverflow.com/a/52756639

0 + veryTiny/2

Example of overflow (this comes up in machine learning and logistic regression type problems). See things like [`logsumexp`](scipy.special.logsumexp) and [`log1p`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.log1p.html?highlight=log1p#scipy.special.log1p)

Let's compute $\log_{10}( 10^a + 10^b )$, which is the log-sum-exp function, and a type of soft-max.  If $b$ is very small, it won't have much contribution, and so $\log_{10}(10^a + 10^b) \approx \log_{10}(10^a) = a$.

However, we can easily run into trouble if we're not careful:

In [ ]:
a = 350.  # if we don't make it 700. (and use 700, the integer, instead, then its OK)
b = 340.
math.log10( 10**a + 10**b )

OverflowError: ignored

In [ ]:
a + math.log10( 10**0 + 10**(b-a) )

350.0000000000434